# Public Transport Payoff Calculation

## Parameters:

- Network Information: A graph with 25 nodes and some edges connecting them.
- Bus routes: There are three bus routes in this network, each route is bi-directional, thus there are 6 bus paths:
    - 'Route1': [1, 5, 6, 7, 12, 15, 16, 17, 18, 20, 21, 14, 10, 9, 3, 2, 1],
    - 'Route2': [1, 2, 3, 9, 10, 14, 21, 20, 18, 17, 16, 15, 12, 7, 6, 5, 1],
    - 'Route3': [25, 24, 23, 22, 14, 19, 13, 8, 4, 5, 4, 8, 13, 19, 14, 22, 23, 24, 25],
    - 'Route4': [16, 12, 11, 7, 4, 3, 4, 7, 11, 12, 16]

## Payoff Calculation:

The payoff of Public Transport includes ticket revenue, fleet investment cost, charging station cost, charging cost, maintence cost, tax cost.

- Step1: Bus scheduling

    According to the route information and travelling demand, first calculate the route for each OD pair under PT system, then calculate the burden on each arc during each time interval, finally calculate the minimal departing bus number for each route in each time interval which satisfy all travelling demand.

- Step2: Payoff Evaluation

    According to the travel demand data, the ticket revenue can be calculated. According to the bus scheduling information, as well as the route length, the minimal fleet size which considers charging time consumpsion can be fiured out. Then the fleet investment cost, charging station cost, charging cost, maintence cost and tax cost can be evaluated.

## HyperParameter

- Travelling Demand:
    - Original Test Case: all_traveling_orders.xlsx
    - 1.75,2.5.xlsx
    - 1.75,3.xlsx
    - 2,2.5.xlsx
    - 2,3.xlsx

- Tax rate
- Bus cost
- Charging pile cost
- unit charging cost
- unit maintenance cost
- Travelling speed
- Charging speed
- Bus capacity
- The system planned operating days


In [47]:
# Import Libraries
import numpy as np
import pandas as pd
import math
import heapq
import networkx as nx
import matplotlib.pyplot as plt

In [48]:
# Parameters for the network
TIME_INTERVALS = 48  # Number of time intervals
NODES = 25  # Number of nodes
ticket_price = 3
operation_hours = 18

# Define routes with stops
routes = {
    'Route1': [1, 5, 6, 7, 12, 15, 16, 17, 18, 20, 21, 14, 10, 9, 3, 2, 1],
    'Route2': [1, 2, 3, 9, 10, 14, 21, 20, 18, 17, 16, 15, 12, 7, 6, 5, 1],
    'Route3': [25, 24, 23, 22, 14, 19, 13, 8, 4, 5, 4, 8, 13, 19, 14, 22, 23, 24, 25],
    'Route4': [16, 12, 11, 7, 4, 3, 4, 7, 11, 12, 16]
}

# Other Parameters
bus_cost = 450000*0.1845 # purchasing cost of a bus
charging_fix_cost = 100000*0.1845 # fix construction cost of one charging station
charging_pile_cost = 1000*0.1845 # construction cost of one charging pile

average_electricity_price_per_KWh = 0.20134
KWh_cost_per_km = 446/350
charging_KWh_per_time_interval = 45
unit_charging_cost = average_electricity_price_per_KWh * KWh_cost_per_km # charging cost per km
charging_speed = charging_KWh_per_time_interval/KWh_cost_per_km # charging kilometers per time interval

unit_operation_cost = 30 # operation cost per vehicle per hour

speed = 20 # travelling kilometers per time interval
C = 70  # Capacity of each bus

n = 365 # The system planned operating days

demand_files = ['1.75,2.5.xlsx', '1.75,3.xlsx', '2,2.5.xlsx', '2,3.xlsx']

In [50]:
# Load the demand data D from Excel file
file_path = '2,2.5.xlsx'  # Replace with the actual path
xl = pd.ExcelFile(file_path)

# Initialize a three-dimensional NumPy array to store the demand data
D = np.empty((len(xl.sheet_names), NODES, NODES))

# Read demand data for all time intervals
for i, sheet in enumerate(xl.sheet_names):
    D[i] = xl.parse(sheet, header=None).values

In [51]:
# Load the distance data d from Excel file
file_path = 'distance.xlsx'  # Replace with the actual path
xl = pd.read_excel(file_path, header=None)

# Transform into Numpy array
d = xl.values

In [52]:
def initialize_graph(routes):
    graph = nx.DiGraph()
    routes_at_stop = {}  # This will map each stop to the routes that serve it
    
    # First, map all stops to their routes
    for route_id, stops in routes.items():
        for stop in stops:
            if stop not in routes_at_stop:
                routes_at_stop[stop] = set()
            routes_at_stop[stop].add(route_id)
    
    # Now create edges
    for route_id, stops in routes.items():
        for i in range(len(stops) - 1):
            stop1, stop2 = stops[i], stops[i + 1]
            graph.add_edge(stop1, stop2, weight=d[stop1-1,stop2-1])  # Travel time between consecutive stops
            # graph.add_edge(stop2, stop1, weight=d[stop1-1,stop2-1])  # Travel time between consecutive stops

    return graph

In [53]:
graph = initialize_graph(routes)

In [54]:
def calculate_traffic_burden(graph, D, time_intervals):
    F_tij = np.zeros((time_intervals, NODES, NODES))
    for t in range(time_intervals):
        for i in range(NODES):
            for j in range(NODES):
                if i != j:
                    path = nx.shortest_path(graph, i+1, j+1, weight='weight')
                    for k in range(len(path) - 1):
                        current_time = (t + math.floor(nx.shortest_path_length(graph, i+1, path[k+1], weight='weight')/speed)) % 48
                        start, end = path[k], path[k + 1]
                        F_tij[current_time, start-1, end-1] += D[t, i, j]
    return F_tij

F_tij = calculate_traffic_burden(graph, D, TIME_INTERVALS)

In [55]:
def calculate_departing_buses(F_tij, routes, d, bus_capacity):
    departing_buses = {route_id: np.zeros(TIME_INTERVALS) for route_id in routes}
    
    for t in range(TIME_INTERVALS):
        for route_id, stops in routes.items():
            demand = np.zeros(len(stops)-1)
            for i in range(len(stops) - 1):
                start, end = stops[i], stops[i + 1]
                travel_time = math.floor(d[stops[0]-1, end-1]/speed)
                demand[i] = F_tij[(t + travel_time)%48, start-1, end-1]

            departing_buses[route_id][t] = math.ceil(np.max(demand) / bus_capacity)
    
    return departing_buses

# Calculate the departing buses
departing_buses = calculate_departing_buses(F_tij, routes, d, C)


In [56]:
def max_buses_on_trip(timetable, trip_time):
    buses_on_trip = np.zeros(TIME_INTERVALS)

    for t in range(TIME_INTERVALS):
        if timetable[t] > 0:
            start_trip = t
            end_trip = t + trip_time
            if end_trip < TIME_INTERVALS:
                buses_on_trip[start_trip:end_trip] += timetable[t]
            else:
                buses_on_trip[start_trip:] += timetable[t]
    
    return np.max(buses_on_trip)

def max_buses_charging(timetable, trip_time, charging_time):
    buses_charging = np.zeros(TIME_INTERVALS)

    for t in range(TIME_INTERVALS):
        end_trip = t + trip_time
        if end_trip < TIME_INTERVALS and timetable[t] > 0:
            start_charging = end_trip
            end_charging = end_trip + charging_time
            if end_charging < TIME_INTERVALS:
                buses_charging[start_charging:end_charging] += timetable[t]
            else:
                buses_charging[start_charging:] += timetable[t]

    return np.max(buses_charging)

def calculate_fleet_size(departing_buses, routes, d):
    buses_needed = np.zeros(len(routes))
    buses_charging = np.zeros(len(routes))

    for j, (route_id, timetable) in enumerate(departing_buses.items()):
        stops = routes[route_id]
        route_length = sum([d[stops[i]-1, stops[i+1]-1] for i in range(len(stops) - 1)])
        trip_time = math.ceil(route_length/speed)
        charging_time = math.ceil(route_length/charging_speed)
        trip = max_buses_on_trip(timetable, trip_time)
        charging = max_buses_charging(timetable, trip_time, charging_time)
        buses_needed[j] = trip + charging
        buses_charging[j] = charging
    
    return buses_needed, buses_charging

def calculate_route_length(departing_buses, routes, d):
    route_travel_distance = np.zeros(len(routes))

    for j, (route_id, timetable) in enumerate(departing_buses.items()):
        stops = routes[route_id]
        route_length = sum([d[stops[i]-1, stops[i+1]-1] for i in range(len(stops) - 1)])
        route_travel_distance[j] = sum(timetable) * route_length
    
    return route_travel_distance

In [57]:
# Ticket Revenue

U_ticket = np.sum(D) * ticket_price * n
print("ticket revenue", U_ticket)

# Fleet Investment
fleet_size, charging_fleet_size = calculate_fleet_size(departing_buses, routes, d)
fleet_investment_cost = bus_cost * sum(fleet_size)
print("fleet investment cost", fleet_investment_cost)

# Charging Station Investment
charging_station_cost = 3 * charging_fix_cost + charging_pile_cost * sum(charging_fleet_size)
print("charging station cost", charging_station_cost)

# Charging cost
total_travel_distance = sum(calculate_route_length(departing_buses, routes, d))
charging_cost = total_travel_distance * unit_charging_cost * n
print("charging cost", charging_cost)

# Maintenance cost
operation_cost = sum(fleet_size) * operation_hours * unit_operation_cost * n
print("operation cost", operation_cost)

Payoff = U_ticket - fleet_investment_cost - charging_station_cost - charging_cost - operation_cost

# Tax Cost
Payoff = (1 - tax_rate) * Payoff

print("Payoff", Payoff)

print("Net Payoff", U_ticket - charging_cost - operation_cost)

ticket revenue 20939685.0
fleet investment cost 8053425.0
charging station cost 61992.0
charging cost 3106616.0626182854
operation cost 19118700.0
Payoff -9401048.062618285
Net Payoff -1285631.0626182854


In [58]:
fleet_size

array([33., 28., 28.,  8.])

In [59]:
charging_fleet_size

array([12., 11., 10.,  3.])

The following code iterates over the four cases and calculates the payoffs and the net payoffs which ignores infrastructure investments.

In [2]:
import numpy as np
import pandas as pd
import math
import networkx as nx
import matplotlib.pyplot as plt

# Parameters for the network
TIME_INTERVALS = 48  # Number of time intervals
NODES = 25  # Number of nodes
ticket_price = 3
operation_hours = 18

# Define routes with stops
routes = {
    'Route1': [1, 5, 6, 7, 12, 15, 16, 17, 18, 20, 21, 14, 10, 9, 3, 2, 1],
    'Route2': [1, 2, 3, 9, 10, 14, 21, 20, 18, 17, 16, 15, 12, 7, 6, 5, 1],
    'Route3': [25, 24, 23, 22, 14, 19, 13, 8, 4, 5, 4, 8, 13, 19, 14, 22, 23, 24, 25],
    'Route4': [16, 12, 11, 7, 4, 3, 4, 7, 11, 12, 16]
}

# Other Parameters
tax_rate = 0  # tax rate

bus_cost = 450000 * 0.1845  # purchasing cost of a bus
charging_fix_cost = 100000 * 0.1845  # fix construction cost of one charging station
charging_pile_cost = 1000 * 0.1845  # construction cost of one charging pile
average_electricity_price_per_KWh = 0.20134
KWh_cost_per_km = 446 / 350
charging_KWh_per_time_interval = 45
unit_charging_cost = average_electricity_price_per_KWh * KWh_cost_per_km  # charging cost per km
unit_operation_cost = 30  # operation cost per vehicle per hour

speed = 20  # travelling kilometers per time interval
charging_speed = charging_KWh_per_time_interval / KWh_cost_per_km  # charging kilometers per time interval
C = 70  # Capacity of each bus

n = 365  # The system planned operating days

# List of demand files
demand_files = ['1.75,2.5.xlsx', '1.75,3.xlsx', '2,2.5.xlsx', '2,3.xlsx']

# Load the distance data d from Excel file
file_path = 'distance.xlsx'  # Replace with the actual path
xl = pd.read_excel(file_path, header=None)

# Transform into Numpy array
d = xl.values

def initialize_graph(routes):
    graph = nx.DiGraph()
    routes_at_stop = {}  # This will map each stop to the routes that serve it
    
    # First, map all stops to their routes
    for route_id, stops in routes.items():
        for stop in stops:
            if stop not in routes_at_stop:
                routes_at_stop[stop] = set()
            routes_at_stop[stop].add(route_id)
    
    # Now create edges
    for route_id, stops in routes.items():
        for i in range(len(stops) - 1):
            stop1, stop2 = stops[i], stops[i + 1]
            graph.add_edge(stop1, stop2, weight=d[stop1-1, stop2-1])  # Travel time between consecutive stops

    return graph

def calculate_traffic_burden(graph, D, time_intervals):
    F_tij = np.zeros((time_intervals, NODES, NODES))
    for t in range(time_intervals):
        for i in range(NODES):
            for j in range(NODES):
                if i != j:
                    path = nx.shortest_path(graph, i+1, j+1, weight='weight')
                    for k in range(len(path) - 1):
                        current_time = (t + math.floor(nx.shortest_path_length(graph, i+1, path[k+1], weight='weight') / speed)) % 48
                        start, end = path[k], path[k + 1]
                        F_tij[current_time, start-1, end-1] += D[t, i, j]
    return F_tij

def calculate_departing_buses(F_tij, routes, d, bus_capacity):
    departing_buses = {route_id: np.zeros(TIME_INTERVALS) for route_id in routes}
    
    for t in range(TIME_INTERVALS):
        for route_id, stops in routes.items():
            demand = np.zeros(len(stops)-1)
            for i in range(len(stops) - 1):
                start, end = stops[i], stops[i + 1]
                travel_time = math.floor(d[stops[0]-1, end-1] / speed)
                demand[i] = F_tij[(t + travel_time) % 48, start-1, end-1]

            departing_buses[route_id][t] = math.ceil(np.max(demand) / bus_capacity)
    
    return departing_buses

def max_buses_on_trip(timetable, trip_time):
    buses_on_trip = np.zeros(TIME_INTERVALS)

    for t in range(TIME_INTERVALS):
        if timetable[t] > 0:
            start_trip = t
            end_trip = t + trip_time
            if end_trip < TIME_INTERVALS:
                buses_on_trip[start_trip:end_trip] += timetable[t]
            else:
                buses_on_trip[start_trip:] += timetable[t]
    
    return np.max(buses_on_trip)

def max_buses_charging(timetable, trip_time, charging_time):
    buses_charging = np.zeros(TIME_INTERVALS)

    for t in range(TIME_INTERVALS):
        end_trip = t + trip_time
        if end_trip < TIME_INTERVALS and timetable[t] > 0:
            start_charging = end_trip
            end_charging = end_trip + charging_time
            if end_charging < TIME_INTERVALS:
                buses_charging[start_charging:end_charging] += timetable[t]
            else:
                buses_charging[start_charging:] += timetable[t]

    return np.max(buses_charging)

def calculate_fleet_size(departing_buses, routes, d):
    buses_needed = np.zeros(len(routes))
    buses_charging = np.zeros(len(routes))

    for j, (route_id, timetable) in enumerate(departing_buses.items()):
        stops = routes[route_id]
        route_length = sum([d[stops[i]-1, stops[i+1]-1] for i in range(len(stops) - 1)])
        trip_time = math.ceil(route_length / speed)
        charging_time = math.ceil(route_length / charging_speed)
        trip = max_buses_on_trip(timetable, trip_time)
        charging = max_buses_charging(timetable, trip_time, charging_time)
        buses_needed[j] = trip + charging
        buses_charging[j] = charging
    
    return buses_needed, buses_charging

def calculate_route_length(departing_buses, routes, d):
    route_travel_distance = np.zeros(len(routes))

    for j, (route_id, timetable) in enumerate(departing_buses.items()):
        stops = routes[route_id]
        route_length = sum([d[stops[i]-1, stops[i+1]-1] for i in range(len(stops) - 1)])
        route_travel_distance[j] = sum(timetable) * route_length
    
    return route_travel_distance

# Load the distance data d from Excel file
file_path = 'distance.xlsx'  # Replace with the actual path
xl = pd.read_excel(file_path, header=None)

# Transform into Numpy array
d = xl.values

graph = initialize_graph(routes)

for file_path in demand_files:
    print(f"\nProcessing file: {file_path}")

    # Load the demand data D from Excel file
    xl = pd.ExcelFile(file_path)

    # Initialize a three-dimensional NumPy array to store the demand data
    D = np.empty((len(xl.sheet_names), NODES, NODES))

    # Read demand data for all time intervals
    for i, sheet in enumerate(xl.sheet_names):
        D[i] = xl.parse(sheet, header=None).values

    F_tij = calculate_traffic_burden(graph, D, TIME_INTERVALS)
    departing_buses = calculate_departing_buses(F_tij, routes, d, C)

    # Ticket Revenue
    U_ticket = np.sum(D) * ticket_price * n
    print("Ticket revenue:", U_ticket)

    # Fleet Investment
    fleet_size, charging_fleet_size = calculate_fleet_size(departing_buses, routes, d)
    fleet_investment_cost = bus_cost * sum(fleet_size)
    print("Fleet investment cost:", fleet_investment_cost)

    # Charging Station Investment
    charging_station_cost = 3 * charging_fix_cost + charging_pile_cost * sum(charging_fleet_size)
    print("Charging station cost:", charging_station_cost)

    # Charging cost
    total_travel_distance = sum(calculate_route_length(departing_buses, routes, d))
    charging_cost = total_travel_distance * unit_charging_cost * n
    print("Charging cost:", charging_cost)

    # Maintenance cost
    operation_cost = sum(fleet_size) * operation_hours * unit_operation_cost * n
    print("Operation cost:", operation_cost)

    Payoff = U_ticket - fleet_investment_cost - charging_station_cost - charging_cost - operation_cost

    # Tax Cost
    Payoff = (1 - tax_rate) * Payoff

    print("Payoff:", Payoff)
    print("Net Payoff:", U_ticket - charging_cost - operation_cost)
    print("Fleet size:", fleet_size)
    print("Charging fleet size:", charging_fleet_size)



Processing file: 1.75,2.5.xlsx
Ticket revenue: 17812365.0
Fleet investment cost: 6475950.0
Charging station cost: 60885.0
Charging cost: 2538558.7576251426
Operation cost: 15373800.0
Payoff: -6636828.757625142
Net Payoff: -99993.75762514211
Fleet size: [26. 22. 22.  8.]
Charging fleet size: [10.  8.  9.  3.]

Processing file: 1.75,3.xlsx
Ticket revenue: 17588985.0
Fleet investment cost: 6309900.0
Charging station cost: 60700.5
Charging cost: 2529943.315478857
Operation cost: 14979600.0
Payoff: -6291158.815478858
Net Payoff: 79441.6845211424
Fleet size: [24. 22. 22.  8.]
Charging fleet size: [9. 8. 9. 3.]

Processing file: 2,2.5.xlsx
Ticket revenue: 20939685.0
Fleet investment cost: 7389225.0
Charging station cost: 61438.5
Charging cost: 2840099.2327451427
Operation cost: 17541900.0
Payoff: -6892977.732745143
Net Payoff: 557685.7672548592
Fleet size: [33. 22. 26.  8.]
Charging fleet size: [12.  8. 10.  3.]

Processing file: 2,3.xlsx
Ticket revenue: 19016865.0
Fleet investment cost: 722